# 2. Karakterkódolás

Számítógépes nyelvészet, 2018 tavasz

Simon Eszter, Mittelholcz Iván

MTA, Nyelvtudományi Intézet

## Tartalom

* [1. Szöveges fájlok](#1.-Sz%C3%B6veges-f%C3%A1jlok)
    + [1.1. POSIX definíció](#1.1.-POSIX-definíció)
    + [1.2. Magyarázatok](#1.2.-Magyarázatok)
* [2. Karakterkódolások](#2.-Karakterk%C3%B3dol%C3%A1sok)
    + [2.1. ASCII](#2.1.-ASCII)
    + [2.2. Kiterjesztett ASCII](#2.2.-Kiterjesztett-ASCII)
    + [2.3. Unicode](#2.3.-Unicode)
* [3. Tanulságok](#3.-Tanulságok)
    * [3.1. Parancssor](#3.1.-Parancssor)
* [4. Python](#4.-Python)
* [5. Irodalom](#5.-Irodalom)

## 1. Szöveges fájlok

### 1.1. [POSIX definíció](http://pubs.opengroup.org/onlinepubs/9699919799/basedefs/V1_chap03.html#tag_03_403)
> A file that contains characters organized into zero or more lines.
The lines do not contain NUL characters and none can exceed {LINE\_MAX} bytes in length,
including the \<newline\> character. Although POSIX.1-2008 does not distinguish between text files and binary files (see the ISO C standard), many utilities only produce predictable or meaningful output when operating on text files.

### 1.2. Magyarázatok

__POSIX__: Portable Operating System Interface for uniX

__karakter__: betűk, számok, írásjelek, nem nyomtatható (vezérlő)jelek

__sor__
* line feed (LF); carriage return (CR) (Na de milyen karakterkódolásban?)
* oprendszer függő
    * Windows: CR+LF
    * *nix: LF
    * classic MacOS: CR

In [ ]:
%%bash
# Hány sor van egy szövegben?
echo -n 'alma' | wc -l
echo 'alma' | wc -l
echo -e 'alma\n' | wc -l

__NUL-karakter__: C-stringek záró eleme; null-bájt (00000000)

__LINE_MAX__: A _limits.h_ header-ben definiált érték egy sor
maximális hosszára, bájtban (C)

#### Miért van különbség szöveges és bináris fájlok között?

In [66]:
src = '''#include<stdio.h>

int main()
{
    printf("Hello World\\n");
    return 0;
}
'''
print(src, file=open('hello.c', 'w'))

In [67]:
%%bash
cat hello.c

#include<stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}



In [68]:
%%bash
gcc hello.c
cat -A a.out

^?ELF^B^A^A^@^@^@^@^@^@^@^@^@^C^@>^@^A^@^@^@0^E^@^@^@^@^@^@@^@^@^@^@^@^@^@(^Y^@^@^@^@^@^@^@^@^@^@@^@8^@^I^@@^@^]^@^\^@^F^@^@^@^E^@^@^@@^@^@^@^@^@^@^@@^@^@^@^@^@^@^@@^@^@^@^@^@^@^@M-x^A^@^@^@^@^@^@M-x^A^@^@^@^@^@^@^H^@^@^@^@^@^@^@^C^@^@^@^D^@^@^@8^B^@^@^@^@^@^@8^B^@^@^@^@^@^@8^B^@^@^@^@^@^@^\^@^@^@^@^@^@^@^\^@^@^@^@^@^@^@^A^@^@^@^@^@^@^@^A^@^@^@^E^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@8^H^@^@^@^@^@^@8^H^@^@^@^@^@^@^@^@ ^@^@^@^@^@^A^@^@^@^F^@^@^@M-8^M^@^@^@^@^@^@M-8^M ^@^@^@^@^@M-8^M ^@^@^@^@^@X^B^@^@^@^@^@^@`^B^@^@^@^@^@^@^@^@ ^@^@^@^@^@^B^@^@^@^F^@^@^@M-H^M^@^@^@^@^@^@M-H^M ^@^@^@^@^@M-H^M ^@^@^@^@^@M-p^A^@^@^@^@^@^@M-p^A^@^@^@^@^@^@^H^@^@^@^@^@^@^@^D^@^@^@^D^@^@^@T^B^@^@^@^@^@^@T^B^@^@^@^@^@^@T^B^@^@^@^@^@^@D^@^@^@^@^@^@^@D^@^@^@^@^@^@^@^D^@^@^@^@^@^@^@PM-etd^D^@^@^@M-p^F^@^@^@^@^@^@M-p^F^@^@^@^@^@^@M-p^F^@^@^@^@^@^@<^@^@^@^@^@^@^@<^@^@^@^@^@^@^@^D^@^@^@^@^@^@^@QM-etd^F^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^@^P^@^@^@^@^@^@^@

#### Miért nincs különbség szöveges és bináris fájlok között?

In [ ]:
%%bash
xxd -b hello.c

In [ ]:
%%bash
xxd -b a.out

## 2. Karakterkódolások

Karakterkódolás: a karakterkészlet („ábécé”) és a bitek közti
kapcsolatot leírása.

* nem minden számít külön karakternek
  _<br/>A dőlt, félkövér, ligatúrák, alsó/felső index,
  stb karakterek sokszor csak a fontkészletekben vannak._
* nem kell minden lehetséges bitsorozathoz karaktert rendelni
  _<br/>Pl. az ASCII nem rendel karaktert az 1-el kezdődő bájtokhoz (a $11111111$ nem jelent semmit)_
* nem kell minden lehetséges karakterhez bitsorozatot rendelni
  _<br/>Pl. az ASCII nem kódolja a cirill karaktereket._

__encoding__: $\{String\} \to \{Bitsorozat\}$

Példa: $\ ABC \to 01000001\ 01000010\ 01000011$

Alkalmazás: pl. szövegbevitel

__decoding__: $\{Bitsorozat\} \to \{String\}$

Példa: $\ 01000001\ 01000010\ 01000011 \to ABC$

Alkalmazás: pl. szövegmegjelenítés

### 2.1. ASCII

* 1963, USA: ékezet nélküli, angol karakterek
* távírókhoz: vezérlőkarakterek, pl [CR](https://youtu.be/Pp2IJIk5qvc), LF, ESC, BELL
* 128 karkter – 7 bit (0-tól 127-ig számozva)
* ahol 1 bájt = 8 bit: ’0’ prefix minden ASCII-kód elé

<img src='https://upload.wikimedia.org/wikipedia/commons/c/cf/USASCII_code_chart.png' title='ASCII karaktertábla' />

#### 2.1.1. Bitenkénti operátorok, letter case

In [17]:
bin(0b1000001 | 0b0100000)

'0b1100001'

In [19]:
bin(0b1100001 ^ 0b0100000)

'0b1000001'

In [20]:
chr(65 | 32)

'a'

In [21]:
ord('a')

97

In [49]:


print(ascii_upper('1'))
print(ascii_upper('`'))
print(ascii_upper('a'))
print(ascii_upper('g'))
print(ascii_upper('z'))
print(ascii_upper('{'))

1
`
A
G
Z
{


##### Feladat

A fentiek segítségével írjunk olyan python függvényt,
* ami kisbetűsíti az ascii nagybetűket, minden mást érintetlenül hagy (ascii_lower),
* ami nagybetűsíti az ascii kisbetűket, minden mást érintetlenül hagy (ascii_upper).

Használat:

```python
>>> ascii_lower('U')
'u'
>>> ascii_lower('Ű')
'Ű'
>>> ascii_upper('g')
'G'
>>> ascii_upper('!')
'!'
```

Módosítsuk a függvényeinket, hogy ne csak egy-egy karakterre, de hosszabb stringekre is működjenek!

<font color=white>

def ascii_upper(char):
    res = char
    if 'a' <= char <= 'z':
        res = chr(ord(char) ^ 32)
    return res

def ascii_upper(char):
    res = char
    if isinstance(char, str) and len(char) == 1 and 'a' <= char <= 'z':
        res = chr(ord(char) ^ 32)
    return res

def ascii_upper(char):
    res = char
    try:
        if 'a' <= char <= 'z':
            res = chr(ord(char) ^ 32)
    except TypeError as err:
        pass
    return res

print(ascii_upper('g'))
print(ascii_upper('ű'))
print(ascii_upper('!'))
print(ascii_upper('1'))
print(ascii_upper(1))
print(ascii_upper('alma'))

#</font>


In [52]:
ascii_upper('b')

'B'

In [54]:
ascii_upper('ű')

'ű'

In [3]:
type('b')

str

In [11]:
ord??

### 2.2. Kiterjesztett ASCII

Van egy maradék bit, használjuk ki!

* [ISO 8859](https://en.wikipedia.org/wiki/ISO/IEC_8859#Table) (1983)
    - ISO 8859-1: nyugat-európai
    - ISO 8859-2: közép-európai
    - ISO 8859-5/6/7/8: cirill / arab / görög / héber
* [Windows code pages](https://en.wikipedia.org/wiki/Windows_code_page) (1985)
    - Windows-1250: közép- és kelet-európai
    - Windows-1252: nyugat-európai

Summa:

* fix, 1-bájtos kódolások
* több külön lapon lévő nyelv használata egy dokumentumban probléma (többnyelvű lapok)
* minden bájtsorozat érvényes (lehetne) és dekódolható (lehetne) bármelyik kódolással

### 2.3. Unicode

* 1988, egy ideig vele párhuzamosan fut az Universal Coded Character Set (UCS, 1989), ami később beolvad a Unicode-ba
* cél minden írásrendszer karaktereit egy egységes kódolásban kezelni
* két lépcsős felépítés:
    * Unicode: karakter $\to$ kódpont
    * karakterkódolás: kódpont $\to$ bitek (ebből több van)    

#### 3.2.1. Kódpontok

* jelölés: U+XXXX (négy-hat jegyű hexakód, pl. 'A' = U+0041)
* 17 lap, egyenként $2^{16}$ hellyel (kb. 1.100.000 hely)
    * 0\. lap: [Basic Multilingual Plane](https://upload.wikimedia.org/wikipedia/commons/8/8e/Roadmap_to_Unicode_BMP.svg)
    * 2., 3. és 14. lap: kiegészító lapok
    * 4-13. lapok: nem használtak (még)
    * 15-16. lapok: saját használatra
* Unicode 10.0 (2017): kb 137.000 foglalt (kb 12%)

* több lehetséges karakterkódolás, ami a kódpontokat bitekre képezi
* állandó hosszúságú (tömb, indexelhetőség, string-hossz)
* változó hosszúságú (a fentiek mind nem)

#### 3.2.2. UTF és UTC karakterkódolások

* több lehetséges karakterkódolás, ami a kódpontokat bitekre képezi
    * UCS: _Universal Character Set_, beolvadt a Unicode-ba
    * UTF: _Unicode Transformation Format_
* állandó hosszúságú (tömb, indexelhetőség, string-hossz)
* változó hosszúságú (a fentiek mind nem)

__UCS-4 / UTF-32__

* állandó hosszúságú, 4 bájt
* sok hely: kb 2,1 milliárd ($2^{31}$ -- előjelbit nem használt)
* hátrányok:
    * pazarló
    * NUL-bájtok a szövegben

__UCS-2__

* állandó hosszúságú, 2 bájt
* kevés hely: kb 65 ezer ($2^{16}$)
* egy az egyben kódolja a BMP-t
* szokás keverni a UTF-16-tal, de nem azonos vele (részhalmaz)

__UTF-16__

* változó hosszúságú, 2 vagy 4 bájt
* 2 bájton egy az egyben kódolja a BMP-t
* 4 bájton a Unicode többi részét ([hogyan?](https://en.wikipedia.org/wiki/UTF-16#Description))
* nem minden bájtsorozat jólformált, az egy-, első- és második-bájtok mind különböznek

__UTF-8__

* változó hosszúságú, 1-4 bájt
* 1 bájton egy az egyben kódolja a ASCII-t
* 2-4 bájton a Unicode többi részét ([hogyan?](https://en.wikipedia.org/wiki/UTF-8#Description))
* nem minden bájtsorozat jólformált, az egy-, kezdő- és folytató-bájtok mind különböznek

<img src='https://upload.wikimedia.org/wikipedia/commons/c/c4/Utf8webgrowth.svg'>

### 3. Tanulságok

Nincsen _plain text_, kódolás nélkül.

HTML példa: Honnan tudja a böngésző, hogy dekódolja a HTML header-t?

```html
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
</head>
```



Mi alapján lehet tippelni?

#### 3.1. Parancssor

In [31]:
%%bash

file hello.c
echo
file a.out

hello.c: C source, ASCII text

a.out: ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 3.2.0, BuildID[sha1]=7750ec600447355c1bd505fbac9fbe69c06c72d4, not stripped


In [32]:
%%bash
# -i kapcsoló: MIME típus (Multipurpose Internet Mail Extensions)

file -i hello.c
echo
file -i a.out

hello.c: text/x-c; charset=us-ascii

a.out: application/x-sharedlib; charset=binary


In [55]:
%%bash
# Miért működik?

iconv -f utf-8 -t latin1 hello.c

#include<stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}



In [33]:
%%bash
# Mi a baj?

echo 'árvíztűrő' | iconv -f utf-8 -t latin1

�rv�zt

iconv: illegal input sequence at position 8


In [60]:
%%bash
# Mi a jó?
# Mi a baj?

echo 'árvíztûrõ' | iconv -f utf-8 -t latin1
echo 'árvíztűrő' | iconv -f utf-8 -t latin2
echo '�' # U+FFFD REPLACEMENT CHARACTER

�rv�zt�r�
�rv�zt�r�
�


In [61]:
%%bash
# A tippelés korlátai.

echo 'árvíztűrő' | iconv -f utf-8 -t iso-8859-2 >latin2.txt
file latin2.txt
echo
file -i latin2.txt

latin2.txt: ISO-8859 text

latin2.txt: text/plain; charset=iso-8859-1


##### További hasznos parancsok, csomagok

* `recode`: hasonló az `iconv`-hoz, konvertál különböző kódolások között
* `ascii`: ASCII karakterekről infó, karaktertábla
* [Unicode Utilities](http://billposer.org/Software/unidesc.html): Unicode karakterekről infók, több parancsot tartalmazó csomag
* vim
    * `:as[cii]`
    * `ga`
    * [vim-characterize](https://github.com/tpope/vim-characterize)

### 4. Python

### 5. Irodalom

* [Character sets, encodings, and Unicode](http://www.gammon.com.au/unicode/)
* [The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Character Sets](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)
* [Python: Unicode HOWTO](https://docs.python.org/3/howto/unicode.html)
* [The Unicode® Standard: A Technical Introduction](http://www.unicode.org/standard/principles.html)
* [Latest Version of the Unicode Standard](http://www.unicode.org/versions/latest/)

***

In [72]:
%%bash
# takarítás

rm -f hello.c
rm -f a.out
rm -f latin2.txt